# Setup:

In [1]:
import pandas as pd
import numpy as np
import re

from pyathena import connect
try:
    from pyathena.pandas_cursor import PandasCursor
except:
    from pyathena.pandas.cursor import PandasCursor

In [2]:
def run_query(sql_query: str):
    """Run a data definition query on Athena

    Args:
        sql_query (str): SQL query to be run on Athena.
    """

    cursor = connect(
        s3_staging_dir="s3://3efee1-analytics-athena-results/output/",
        region_name="us-east-2",
    ).cursor(PandasCursor)
    return cursor.execute(sql_query).as_pandas()

# Get All Data - Whole Time Range Availabe

In [3]:
query_bingo_aloha = """
WITH first_logins     AS (
    SELECT user_id
         , MIN(meta_date) first_login
    FROM etl__century_games_ncmgu__bingo_aloha_r3g9v.session_start
    GROUP BY user_id
    )
   , logins           AS (
    SELECT user_id
         , meta_date
         , first_login
         , test_group
    FROM etl__century_games_ncmgu__bingo_aloha_r3g9v.session_start
         INNER JOIN console.clean_abtest USING (user_id)
         LEFT JOIN  first_logins USING (user_id)
    WHERE meta_company_id = 'century-games-ncmgu'
      AND meta_project_id = 'bingo-aloha-r3g9v'
      AND first_login >= DATE '2022-07-01'
    )
   , player_spend     AS (
    SELECT test_group
         , user_id
         , meta_date
         , SUM(payments.sum_bi_payment_amount_usd) spend
    FROM etl__century_games_ncmgu__bingo_aloha_r3g9v.bi_payment payments
         INNER JOIN console.clean_abtest                        abtest USING (user_id)
    WHERE meta_company_id = 'century-games-ncmgu'
      AND meta_project_id = 'bingo-aloha-r3g9v'
    GROUP BY user_id
           , meta_date
           , test_group
    )
   , filtered_players AS (
    SELECT test_group
         , user_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM player_spend
    )
   , wins_spend_table AS (
    SELECT user_id
         , meta_date
         , filtered_players.test_group
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (player_spend ps
         INNER JOIN filtered_players USING (user_id, meta_date))
    )
   , t_out            AS (
    SELECT user_id
         , test_group
         , SUM(spend)       total_spend
         , SUM(wins_spend)  total_wins_spend
    FROM logins
         LEFT JOIN wins_spend_table USING (user_id, meta_date, test_group)
    GROUP BY user_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [4]:
query_terra_genesis = """
WITH first_logins               AS (
    SELECT account_id
         , MIN(meta_date) first_login
    FROM etl__tilting_point_mjs4k__terragenesis_m89uz.session_start
    GROUP BY account_id
    )
   , logins                     AS (
    SELECT account_id
         , meta_date
         , MIN(first_login) first_login
    FROM etl__tilting_point_mjs4k__terragenesis_m89uz.session_start
         LEFT JOIN first_logins USING (account_id)
    WHERE (first_login >= DATE '2022-07-06')
    GROUP BY account_id
           , meta_date
    )
   , spend_table                AS (
    SELECT account_id
         , meta_date
         , SUM(sum_value) spend
    FROM (etl__tilting_point_mjs4k__terragenesis_m89uz.purchase
        LEFT JOIN first_logins USING (account_id)
             )
    WHERE ((first_login >= DATE '2022-07-06') AND (meta_date >= DATE '2022-07-06'))
    GROUP BY account_id
           , meta_date
    )
   , filtered_players           AS (
    SELECT account_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date) percentile
    FROM spend_table
    )
   , wins_spend_table           AS (
    SELECT account_id
         , meta_date
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (spend_table ps
         INNER JOIN filtered_players USING (account_id, meta_date))
    )
   , date_purchase_given_bucket AS (
    SELECT account_id
         , meta_date
         , compliance_area
         , manufacturer
         , region_tier
         , device_tier
         , predicted_value
         , group_tag
         , spend
         , wins_spend
    FROM logins
         RIGHT JOIN analytics__tilting_point_mjs4k__terragenesis_m89uz.bucket_assignment_map USING (account_id)
         LEFT JOIN  wins_spend_table USING (account_id, meta_date)
    WHERE (meta_date >= DATE '2022-07-06')
    )
   , t_out                      AS (
    SELECT account_id      user_id
         , group_tag       test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM date_purchase_given_bucket
    GROUP BY account_id
           , group_tag
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [5]:
query_spongebob = """
WITH first_logins               AS (
    SELECT account_id
         , MIN(meta_date) first_login
    FROM etl__tilting_point_mjs4k__spongebob_x7d9q.session_start
    GROUP BY account_id
    )
   , logins                     AS (
    SELECT account_id
         , meta_date
    FROM (etl__tilting_point_mjs4k__spongebob_x7d9q.session_start
        LEFT JOIN first_logins USING (account_id)
             )
    WHERE (first_login >= DATE '2022-07-01')
    GROUP BY account_id
           , meta_date
    )
   , spend_table                AS (
    SELECT account_id
         , meta_date
         , SUM(purchase_sum_value) spend
    FROM (etl__tilting_point_mjs4k__spongebob_x7d9q.purchase
        LEFT JOIN first_logins USING (account_id)
             )
    WHERE (first_login >= DATE '2022-07-01')
    GROUP BY account_id
           , meta_date
    )
   , filtered_players           AS (
    SELECT account_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date) percentile
    FROM spend_table
    )
   , wins_spend_table           AS (
    SELECT account_id
         , meta_date
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (spend_table ps
         INNER JOIN filtered_players USING (account_id, meta_date))
    )
   , date_purchase_given_bucket AS (
    SELECT account_id
         , meta_date
         , compliance_area
         , is_restricted_country
         , manufacturer
         , region_tier
         , device_tier
         , predicted_value
         , group_tag
         , spend
         , wins_spend
    FROM logins
         RIGHT JOIN analytics__tilting_point_mjs4k__spongebob_x7d9q.bucket_assignment_map USING (account_id)
         LEFT JOIN  wins_spend_table USING (account_id, meta_date)
    WHERE (meta_date >= DATE '2022-07-01')
    )
   , t_out                      AS (
    SELECT account_id      user_id
         , group_tag       test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM date_purchase_given_bucket
    GROUP BY account_id
           , group_tag
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [6]:
query_ultimex = """
WITH first_logins     AS (
    SELECT user_id
         , MIN(meta_date) first_login
    FROM etl__sparkgaming_vjv6s__ultimate_x_poker_rib6t.session_start logins
    GROUP BY user_id
    )
   , targeted_offers  AS (
    SELECT payments.user_id   user_id
         , offer_type
         , payments.meta_date meta_date
         , payments.sum_value sum_value
    FROM etl__sparkgaming_vjv6s__ultimate_x_poker_rib6t.purchase payments
         LEFT JOIN console."sparkgaming_ultimate-x-poker_offer_store" offers
                   ON (payments.event_params_item_id = offers.product_id)
    )
   , player_spend     AS (
    SELECT user_id
         , test_group
         , offer_type
         , meta_date
         , SUM(sum_value) spend
    FROM targeted_offers
         INNER JOIN console.clean_abtest abtest USING (user_id)
         LEFT JOIN first_logins USING (user_id)
    WHERE (meta_company_id = 'sparkgaming-vjv6s')
      AND (meta_project_id = 'ultimate-x-poker-rib6t')
    GROUP BY user_id
           , test_group
           , offer_type
           , meta_date
    )
   , filtered_players AS (
    SELECT user_id
         , test_group
         , offer_type
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM player_spend
    )
   , wins_spend_table AS (
    SELECT user_id
         , test_group
         , offer_type
         , meta_date
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (player_spend ps
         INNER JOIN filtered_players USING (user_id, meta_date, test_group, offer_type))
    )
   , login_stats      AS (
    SELECT user_id
         , meta_date
         , test_group
    FROM etl__sparkgaming_vjv6s__ultimate_x_poker_rib6t.session_start
        INNER JOIN console.clean_abtest USING (user_id)
        LEFT JOIN first_logins USING (user_id)
    WHERE (meta_company_id = 'sparkgaming-vjv6s')
      AND (meta_project_id = 'ultimate-x-poker-rib6t')
      AND (meta_date > DATE '2022-07-14')
      AND (first_login > DATE '2022-07-14')
    )
   , t_out            AS (
    SELECT user_id
         , test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM login_stats                t1
         LEFT JOIN wins_spend_table t2 USING (user_id, test_group)
    GROUP BY user_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [7]:
query_knighthood = """
WITH first_logins     AS (
    SELECT userid
         , MIN(meta_date) first_login
    FROM etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_login
    GROUP BY userid
    )
   , purchases        AS (
    SELECT userid
         , SUM(sum_metric_iap_dollars) sum_purchases
         , meta_date
    FROM etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_iap payments
    GROUP BY userid
           , meta_date
    )
   , logins           AS (
    SELECT userid
         , logins.meta_date
         , (CASE
                WHEN (experimentgroup = 0) THEN 'Control'
                WHEN (experimentgroup = 1) THEN 'Revised Deals'
                WHEN (experimentgroup = 2) THEN 'Assetario'
                ELSE 'None' END) test_group
    FROM etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_login                      logins
         RIGHT JOIN etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_assign_ab_group abtest USING (userid)
         LEFT JOIN  first_logins                                                      fr_log USING (userid)
    WHERE first_login >= DATE '2022-03-26'
      AND experiment = 'Deal_Revison_Experiment'
    GROUP BY userid
           , logins.meta_date
           , experimentgroup
    )
   , filtered_players AS (
    SELECT userid
         , test_group
         , COALESCE(purchases.meta_date, logins.meta_date) meta_date
         , approx_percentile(sum_purchases, 9.9E-1)        OVER (PARTITION BY COALESCE(purchases.meta_date, logins.meta_date), test_group) percentile
    FROM logins
         LEFT JOIN purchases USING (userid)
    )
   , wins_spend_table AS (
    SELECT userid
         , test_group
         , meta_date
         , sum_purchases                                                                   spend
         , (CASE WHEN (sum_purchases > percentile) THEN percentile ELSE sum_purchases END) wins_spend
    FROM (purchases ps
         INNER JOIN filtered_players USING (userid, meta_date))
    )
   , t_out            AS (
    SELECT userid          user_id
         , test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM logins                     t1
         LEFT JOIN wins_spend_table t2 USING (userid, test_group)
    GROUP BY userid
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [8]:
query_idle_mafia = """
WITH first_logins      AS (
    SELECT account_id
         , MIN(meta_date) first_login
    FROM etl__century_games_ncmgu__idle_mafia_ecbqb.login_stats logins
    GROUP BY account_id
    )
   , player_spend      AS (
    SELECT test_group
         , account_id
         , meta_date
         , SUM(payments.sum_purchases_package_key_daily) spend
    FROM etl__century_games_ncmgu__idle_mafia_ecbqb.shop_package_key_daily_purchase_popularity_stats payments
         INNER JOIN console.clean_abtest                                                             abtest ON (payments.account_id = abtest.user_id)
    WHERE abtest.meta_project_id = 'idle-mafia-ecbqb'
      AND abtest.meta_company_id = 'century-games-ncmgu'
    GROUP BY account_id
           , meta_date
           , test_group
    )
   , filtered_players  AS (
    SELECT test_group
         , account_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM player_spend
    )
   , wins_spend_table  AS (
    SELECT account_id
         , meta_date
         , filtered_players.test_group
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM player_spend ps
         INNER JOIN filtered_players USING (account_id, meta_date)
    )
   , logins_with_group AS (
    SELECT account_id
         , meta_date
         , test_group
    FROM etl__century_games_ncmgu__idle_mafia_ecbqb.login_stats logins
         INNER JOIN console.clean_abtest                        abtest ON (logins.account_id = abtest.user_id)
         LEFT JOIN  first_logins USING (account_id)
    WHERE abtest.meta_project_id = 'idle-mafia-ecbqb'
      AND abtest.meta_company_id = 'century-games-ncmgu'
      AND first_login >= DATE '2022-01-01'
    )
   , t_out             AS (
    SELECT account_id       user_id
         , test_group
         , SUM(spend)       total_spend
         , SUM(wins_spend)  total_wins_spend
    FROM logins_with_group
         LEFT JOIN wins_spend_table USING (account_id, test_group)
    GROUP BY account_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [9]:
query_homw = """
WITH first_logins     AS (
    SELECT user_id
         , MIN(meta_date) first_login
    FROM etl__tinysoft_a9kwp__heroes_magic_war_h2sln.session_start logins
    GROUP BY user_id
    )
   , logins           AS (
    SELECT user_id
         , log.meta_date
         , ab.test_group
    FROM etl__tinysoft_a9kwp__heroes_magic_war_h2sln.session_start log
         LEFT JOIN console.clean_abtest                            ab USING (user_id)
         LEFT JOIN first_logins                                    fl USING (user_id)
    WHERE meta_company_id = 'tinysoft-a9kwp'
      AND meta_project_id = 'heroes-magic-war-h2sln'
      AND first_login >= DATE '2022-03-18'
    GROUP BY user_id
           , log.meta_date
           , test_group
    )
   , purchases        AS (
    SELECT user_id
         , SUM(sum_in_app_purchase_event_value_in_usd) spend
         , MAX(test_group)                             test_group
         , meta_date
    FROM etl__tinysoft_a9kwp__heroes_magic_war_h2sln.in_app_purchase
         LEFT JOIN console.clean_abtest ab USING (user_id)
    WHERE meta_company_id = 'tinysoft-a9kwp'
      AND meta_project_id = 'heroes-magic-war-h2sln'
    GROUP BY user_id
           , meta_date
    )
   , filtered_players AS (
    SELECT test_group
         , user_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM purchases
    )
   , wins_spend_table AS (
    SELECT user_id
         , meta_date
         , filtered_players.test_group
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM purchases ps
         INNER JOIN filtered_players USING (user_id, meta_date)
    )
   , t_out            AS (
    SELECT user_id
         , test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM logins
         LEFT JOIN wins_spend_table USING (user_id, test_group)
    GROUP BY user_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [10]:
df_bingo_aloha   = run_query(query_bingo_aloha)
df_terra_genesis = run_query(query_terra_genesis)
df_spongebob     = run_query(query_spongebob)
df_ultimex       = run_query(query_ultimex)
df_knighthood    = run_query(query_knighthood)
df_idle_mafia    = run_query(query_idle_mafia)
df_homw          = run_query(query_homw)

In [11]:
df_bingo_aloha

,user_id,test_group,total_spend,total_wins_spend
0,294646641,P,0.000000,0.000000
1,291896160,C,0.000000,0.000000
2,287740401,C,0.000000,0.000000
3,242356532,C,0.000000,0.000000
4,289567171,C,0.000000,0.000000
...,...,...,...,...
279525,289216750,C,0.000000,0.000000
279526,247408332,P,8.018328,8.018328
279527,293122220,P,0.000000,0.000000
279528,293117610,C,0.000000,0.000000


In [12]:
df_bingo_aloha.to_csv(  "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_bingo_aloha.csv")
df_terra_genesis.to_csv("/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_terra_genesis.csv")
df_spongebob.to_csv(    "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_spongebob.csv")
df_ultimex.to_csv(      "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_ultimex.csv")
df_knighthood.to_csv(   "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_knighthood.csv")
df_idle_mafia.to_csv(   "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_idle_mafia.csv")
df_homw.to_csv(         "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_homw.csv")

# Get Last 30 Day Data Data

In [14]:
query_bingo_aloha_30 = """
WITH first_logins     AS (
    SELECT user_id
         , MIN(meta_date) first_login
    FROM etl__century_games_ncmgu__bingo_aloha_r3g9v.session_start
    GROUP BY user_id
    )
   , logins           AS (
    SELECT user_id
         , meta_date
         , first_login
         , test_group
    FROM etl__century_games_ncmgu__bingo_aloha_r3g9v.session_start
         INNER JOIN console.clean_abtest USING (user_id)
         LEFT JOIN  first_logins USING (user_id)
    WHERE meta_company_id = 'century-games-ncmgu'
      AND meta_project_id = 'bingo-aloha-r3g9v'
      AND first_login >= CURRENT_DATE - INTERVAL '32' DAY
    )
   , player_spend     AS (
    SELECT test_group
         , user_id
         , meta_date
         , SUM(payments.sum_bi_payment_amount_usd) spend
    FROM etl__century_games_ncmgu__bingo_aloha_r3g9v.bi_payment payments
         INNER JOIN console.clean_abtest                        abtest USING (user_id)
    WHERE meta_company_id = 'century-games-ncmgu'
      AND meta_project_id = 'bingo-aloha-r3g9v'
    GROUP BY user_id
           , meta_date
           , test_group
    )
   , filtered_players AS (
    SELECT test_group
         , user_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM player_spend
    )
   , wins_spend_table AS (
    SELECT user_id
         , meta_date
         , filtered_players.test_group
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (player_spend ps
         INNER JOIN filtered_players USING (user_id, meta_date))
    )
   , t_out            AS (
    SELECT user_id
         , test_group
         , SUM(spend)       total_spend
         , SUM(wins_spend)  total_wins_spend
    FROM logins
         LEFT JOIN wins_spend_table USING (user_id, meta_date, test_group)
    GROUP BY user_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [15]:
query_terra_genesis_30 = """
WITH first_logins               AS (
    SELECT account_id
         , MIN(meta_date) first_login
    FROM etl__tilting_point_mjs4k__terragenesis_m89uz.session_start
    GROUP BY account_id
    )
   , logins                     AS (
    SELECT account_id
         , meta_date
         , MIN(first_login) first_login
    FROM etl__tilting_point_mjs4k__terragenesis_m89uz.session_start
         LEFT JOIN first_logins USING (account_id)
    WHERE first_login >= CURRENT_DATE - INTERVAL '32' DAY
    GROUP BY account_id
           , meta_date
    )
   , spend_table                AS (
    SELECT account_id
         , meta_date
         , SUM(sum_value) spend
    FROM etl__tilting_point_mjs4k__terragenesis_m89uz.purchase
         LEFT JOIN first_logins USING (account_id)
    GROUP BY account_id
           , meta_date
    )
   , filtered_players           AS (
    SELECT account_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date) percentile
    FROM spend_table
    )
   , wins_spend_table           AS (
    SELECT account_id
         , meta_date
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (spend_table ps
         INNER JOIN filtered_players USING (account_id, meta_date))
    )
   , date_purchase_given_bucket AS (
    SELECT account_id
         , meta_date
         , compliance_area
         , manufacturer
         , region_tier
         , device_tier
         , predicted_value
         , group_tag
         , spend
         , wins_spend
    FROM logins
         RIGHT JOIN analytics__tilting_point_mjs4k__terragenesis_m89uz.bucket_assignment_map USING (account_id)
         LEFT JOIN  wins_spend_table USING (account_id, meta_date)
    )
   , t_out                      AS (
    SELECT account_id      user_id
         , group_tag       test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM date_purchase_given_bucket
    GROUP BY account_id
           , group_tag
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [16]:
query_spongebob_30 = """
WITH first_logins               AS (
    SELECT account_id
         , MIN(meta_date) first_login
    FROM etl__tilting_point_mjs4k__spongebob_x7d9q.session_start
    GROUP BY account_id
    )
   , logins                     AS (
    SELECT account_id
         , meta_date
    FROM (etl__tilting_point_mjs4k__spongebob_x7d9q.session_start
        LEFT JOIN first_logins USING (account_id)
             )
    WHERE first_login >= CURRENT_DATE - INTERVAL '32' DAY
    GROUP BY account_id
           , meta_date
    )
   , spend_table                AS (
    SELECT account_id
         , meta_date
         , SUM(purchase_sum_value) spend
    FROM etl__tilting_point_mjs4k__spongebob_x7d9q.purchase
         LEFT JOIN first_logins USING (account_id)
    GROUP BY account_id
           , meta_date
    )
   , filtered_players           AS (
    SELECT account_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date) percentile
    FROM spend_table
    )
   , wins_spend_table           AS (
    SELECT account_id
         , meta_date
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (spend_table ps
         INNER JOIN filtered_players USING (account_id, meta_date))
    )
   , date_purchase_given_bucket AS (
    SELECT account_id
         , meta_date
         , compliance_area
         , is_restricted_country
         , manufacturer
         , region_tier
         , device_tier
         , predicted_value
         , group_tag
         , spend
         , wins_spend
    FROM logins
         RIGHT JOIN analytics__tilting_point_mjs4k__spongebob_x7d9q.bucket_assignment_map USING (account_id)
         LEFT JOIN  wins_spend_table USING (account_id, meta_date)
    )
   , t_out                      AS (
    SELECT account_id      user_id
         , group_tag       test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM date_purchase_given_bucket
    GROUP BY account_id
           , group_tag
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [17]:
query_ultimex_30 = """
WITH first_logins     AS (
    SELECT user_id
         , MIN(meta_date) first_login
    FROM etl__sparkgaming_vjv6s__ultimate_x_poker_rib6t.session_start logins
    GROUP BY user_id
    )
   , targeted_offers  AS (
    SELECT payments.user_id   user_id
         , offer_type
         , payments.meta_date meta_date
         , payments.sum_value sum_value
    FROM etl__sparkgaming_vjv6s__ultimate_x_poker_rib6t.purchase payments
         LEFT JOIN console."sparkgaming_ultimate-x-poker_offer_store" offers
                   ON (payments.event_params_item_id = offers.product_id)
    )
   , player_spend     AS (
    SELECT user_id
         , test_group
         , offer_type
         , meta_date
         , SUM(sum_value) spend
    FROM targeted_offers
         INNER JOIN console.clean_abtest abtest USING (user_id)
         LEFT JOIN first_logins USING (user_id)
    WHERE (meta_company_id = 'sparkgaming-vjv6s')
      AND (meta_project_id = 'ultimate-x-poker-rib6t')
    GROUP BY user_id
           , test_group
           , offer_type
           , meta_date
    )
   , filtered_players AS (
    SELECT user_id
         , test_group
         , offer_type
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM player_spend
    )
   , wins_spend_table AS (
    SELECT user_id
         , test_group
         , offer_type
         , meta_date
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM (player_spend ps
         INNER JOIN filtered_players USING (user_id, meta_date, test_group, offer_type))
    )
   , login_stats      AS (
    SELECT user_id
         , meta_date
         , test_group
    FROM etl__sparkgaming_vjv6s__ultimate_x_poker_rib6t.session_start
        INNER JOIN console.clean_abtest USING (user_id)
        LEFT JOIN first_logins USING (user_id)
    WHERE (meta_company_id = 'sparkgaming-vjv6s')
      AND (meta_project_id = 'ultimate-x-poker-rib6t')
      AND (first_login >= CURRENT_DATE - INTERVAL '32' DAY)
    )
   , t_out            AS (
    SELECT user_id
         , test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM login_stats                t1
         LEFT JOIN wins_spend_table t2 USING (user_id, test_group)
    GROUP BY user_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [18]:
query_knighthood_30 = """
WITH first_logins     AS (
    SELECT userid
         , MIN(meta_date) first_login
    FROM etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_login
    GROUP BY userid
    )
   , purchases        AS (
    SELECT userid
         , SUM(sum_metric_iap_dollars) sum_purchases
         , meta_date
    FROM etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_iap payments
    GROUP BY userid
           , meta_date
    )
   , logins           AS (
    SELECT userid
         , logins.meta_date
         , (CASE
                WHEN (experimentgroup = 0) THEN 'Control'
                WHEN (experimentgroup = 1) THEN 'Revised Deals'
                WHEN (experimentgroup = 2) THEN 'Assetario'
                ELSE 'None' END) test_group
    FROM etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_login                      logins
         RIGHT JOIN etl__phoenix_games_cd8wx__knighthood_ogh3l.metric_assign_ab_group abtest USING (userid)
         LEFT JOIN  first_logins                                                      fr_log USING (userid)
    WHERE first_login >= DATE '2022-03-26'
      AND experiment = 'Deal_Revison_Experiment'
    GROUP BY userid
           , logins.meta_date
           , experimentgroup
    )
   , filtered_players AS (
    SELECT userid
         , test_group
         , COALESCE(purchases.meta_date, logins.meta_date) meta_date
         , approx_percentile(sum_purchases, 9.9E-1)        OVER (PARTITION BY COALESCE(purchases.meta_date, logins.meta_date), test_group) percentile
    FROM logins
         LEFT JOIN purchases USING (userid)
    )
   , wins_spend_table AS (
    SELECT userid
         , test_group
         , meta_date
         , sum_purchases                                                                   spend
         , (CASE WHEN (sum_purchases > percentile) THEN percentile ELSE sum_purchases END) wins_spend
    FROM (purchases ps
         INNER JOIN filtered_players USING (userid, meta_date))
    )
   , t_out            AS (
    SELECT userid          user_id
         , test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM logins
         LEFT JOIN wins_spend_table USING (userid, test_group)
    GROUP BY userid
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [19]:
query_idle_mafia_30 = """
WITH first_logins      AS (
    SELECT account_id
         , MIN(meta_date) first_login
    FROM etl__century_games_ncmgu__idle_mafia_ecbqb.login_stats logins
    GROUP BY account_id
    )
   , player_spend      AS (
    SELECT test_group
         , account_id
         , meta_date
         , SUM(payments.sum_purchases_package_key_daily) spend
    FROM etl__century_games_ncmgu__idle_mafia_ecbqb.shop_package_key_daily_purchase_popularity_stats payments
         INNER JOIN console.clean_abtest                                                             abtest ON (payments.account_id = abtest.user_id)
    WHERE abtest.meta_project_id = 'idle-mafia-ecbqb'
      AND abtest.meta_company_id = 'century-games-ncmgu'
    GROUP BY account_id
           , meta_date
           , test_group
    )
   , filtered_players  AS (
    SELECT test_group
         , account_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM player_spend
    )
   , wins_spend_table  AS (
    SELECT account_id
         , meta_date
         , filtered_players.test_group
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM player_spend ps
         INNER JOIN filtered_players USING (account_id, meta_date)
    )
   , logins_with_group AS (
    SELECT account_id
         , meta_date
         , test_group
    FROM etl__century_games_ncmgu__idle_mafia_ecbqb.login_stats logins
         INNER JOIN console.clean_abtest                        abtest ON (logins.account_id = abtest.user_id)
         LEFT JOIN  first_logins USING (account_id)
    WHERE abtest.meta_project_id = 'idle-mafia-ecbqb'
      AND abtest.meta_company_id = 'century-games-ncmgu'
      AND first_login >= CURRENT_DATE - INTERVAL '32' DAY
    )
   , t_out             AS (
    SELECT account_id       user_id
         , test_group
         , SUM(spend)       total_spend
         , SUM(wins_spend)  total_wins_spend
    FROM logins_with_group
         LEFT JOIN wins_spend_table USING (account_id, test_group)
    GROUP BY account_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out;
"""

In [20]:
query_homw_30 = """
WITH first_logins     AS (
    SELECT user_id
         , MIN(meta_date) first_login
    FROM etl__tinysoft_a9kwp__heroes_magic_war_h2sln.session_start logins
    GROUP BY user_id
    )
   , logins           AS (
    SELECT user_id
         , log.meta_date
         , ab.test_group
    FROM etl__tinysoft_a9kwp__heroes_magic_war_h2sln.session_start log
         LEFT JOIN console.clean_abtest                            ab USING (user_id)
         LEFT JOIN first_logins                                    fl USING (user_id)
    WHERE meta_company_id = 'tinysoft-a9kwp'
      AND meta_project_id = 'heroes-magic-war-h2sln'
      AND first_login >= CURRENT_DATE - INTERVAL '32' DAY
    GROUP BY user_id
           , log.meta_date
           , test_group
    )
   , purchases        AS (
    SELECT user_id
         , SUM(sum_in_app_purchase_event_value_in_usd) spend
         , MAX(test_group)                             test_group
         , meta_date
    FROM etl__tinysoft_a9kwp__heroes_magic_war_h2sln.in_app_purchase
         LEFT JOIN console.clean_abtest ab USING (user_id)
    WHERE meta_company_id = 'tinysoft-a9kwp'
      AND meta_project_id = 'heroes-magic-war-h2sln'
    GROUP BY user_id
           , meta_date
    )
   , filtered_players AS (
    SELECT test_group
         , user_id
         , meta_date
         , approx_percentile(spend, 9.9E-1) OVER (PARTITION BY meta_date, test_group) percentile
    FROM purchases
    )
   , wins_spend_table AS (
    SELECT user_id
         , meta_date
         , filtered_players.test_group
         , spend
         , (CASE WHEN (spend > percentile) THEN percentile ELSE spend END) wins_spend
    FROM purchases ps
         INNER JOIN filtered_players USING (user_id, meta_date)
    )
   , t_out            AS (
    SELECT user_id
         , test_group
         , SUM(spend)      total_spend
         , SUM(wins_spend) total_wins_spend
    FROM logins
         LEFT JOIN wins_spend_table USING (user_id, test_group)
    GROUP BY user_id
           , test_group
    )
SELECT user_id
     , test_group
     , COALESCE(total_spend, 0)      total_spend
     , COALESCE(total_wins_spend, 0) total_wins_spend
FROM t_out
ORDER BY user_id;
"""

In [21]:
df_bingo_aloha_30   = run_query(query_bingo_aloha_30)
df_terra_genesis_30 = run_query(query_terra_genesis_30)
df_spongebob_30     = run_query(query_spongebob_30)
df_ultimex_30       = run_query(query_ultimex_30)
df_knighthood_30    = run_query(query_knighthood_30)
df_idle_mafia_30    = run_query(query_idle_mafia_30)
df_homw_30          = run_query(query_homw_30)

In [22]:
df_bingo_aloha_30

,user_id,test_group,total_spend,total_wins_spend
0,292816461,C,0.000000,0.000000
1,244688974,C,14.921875,14.921875
2,243889863,C,0.000000,0.000000
3,246252763,P,0.000000,0.000000
4,243879512,C,0.000000,0.000000
...,...,...,...,...
231078,245654083,P,0.000000,0.000000
231079,290833180,P,0.000000,0.000000
231080,299047450,P,0.000000,0.000000
231081,250947262,C,0.000000,0.000000


In [23]:
df_bingo_aloha_30.to_csv(  "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_bingo_aloha_30.csv")
df_terra_genesis_30.to_csv("/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_terra_genesis_30.csv")
df_spongebob_30.to_csv(    "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_spongebob_30.csv")
df_ultimex_30.to_csv(      "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_ultimex_30.csv")
df_knighthood_30.to_csv(   "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_knighthood_30.csv")
df_idle_mafia_30.to_csv(   "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_idle_mafia_30.csv")
df_homw_30.to_csv(         "/Users/PeterNovak/Desktop/Bayesian AB Testing/data/data_homw_30.csv")